In [ ]:
#Import a 311 NYC service request.
#Read or convert the columns ‘Created Date’ and Closed Date’ to datetime datatype and create a new column ‘Request_Closing_Time’ as the time elapsed between request creation and request closing. (Hint: Explore the package/module datetime)
#Provide major insights/patterns that you can offer in a visual format (graphs or tables); at least 4 major conclusions that you can come up with after generic data mining.
#Order the complaint types based on the average ‘Request_Closing_Time’, grouping them for different locations.
#Perform a statistical test for the following:
#Please note: For the below statements you need to state the Null and Alternate and then provide a statistical test to accept or reject the Null Hypothesis along with the corresponding ‘p-value’.

#Whether the average response time across complaint types is similar or not (overall)
#Are the type of complaint or service requested and location related?

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
pd.set_option('max_rows',200)
pd.set_option('max_columns',200)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## 1.Import a 311 NYC service request.

project = pd.read_csv('../input/311-service-requests-nyc/311_Service_Requests_from_2010_to_Present.csv')

In [ ]:
project.head(1) ## Head data of the dataset

In [ ]:
project.isna().sum() ## Finding Na in dataset

In [ ]:
project.describe()  ## Describtion of dataset


In [ ]:
project.drop(['School or Citywide Complaint' , 'Vehicle Type' , 'Taxi Company Borough' , 
         'Taxi Pick Up Location' , 'Garage Lot Name'] , axis = 1 , inplace = True)  ## Dropping some columns which is not necessary

In [ ]:
## 2.Read or convert the columns ‘Created Date’ and Closed Date’ to datetime datatype and create a new column
##‘Request_Closing_Time’ as the time elapsed between request creation and request closing. 
##(Hint: Explore the package/module datetime)

project['Created Date'] =  pd.to_datetime(project['Created Date'])
project['Closed Date'] =  pd.to_datetime(project['Closed Date'])

project['Request_Closing_Time'] = project['Closed Date'] - project['Created Date']
project['Request_Closing_Time_minutes']=project['Request_Closing_Time'].dt.seconds/60
project.head(1)

In [ ]:
## 3.Provide major insights/patterns that you can offer in a visual format (graphs or tables); 
## at least 4 major conclusions that you can come up with after generic data mining.

insight=project.groupby('Agency')['Complaint Type'].value_counts().to_frame().rename(columns={'Complaint Type':'Count'}).sort_values('Count',ascending=False)
insight.reset_index(inplace=True)

## Graph

fig=plt.figure(figsize=(10,8))
plt.grid()
plt.barh(insight['Complaint Type'],insight['Count'],  label='complaint count',
         color='#7FFF00',edgecolor='black', linewidth=1.2, alpha=0.8)
for i, v in enumerate(insight['Count']):
    plt.text(v + 0.2, i + .1, str(v), color='blue', fontweight='bold')

plt.show()

In [ ]:
## 4.Order the complaint types based on the average ‘Request_Closing_Time’, grouping them for different locations.

order = project[['City', 'Complaint Type', 'Request_Closing_Time','Request_Closing_Time_minutes']]
# project['Request_Closing_Time_in_seconds'] = project['Request_Closing_Time'].apply(lambda x : x.seconds)
order.groupby(['City', 'Complaint Type'])['Request_Closing_Time_minutes'].mean().to_frame()

In [ ]:
##  5.Perform a statistical test

from scipy.stats import ttest_ind 
average_response_time = {}
unique_complaint = set(project['Complaint Type'].unique())

project['Request_Closing_Time_in_seconds'] = project['Request_Closing_Time'].apply(lambda x : x.seconds)

for i in unique_complaint:
    average_response_time[i] = project[project['Complaint Type'] == i]['Request_Closing_Time_in_seconds'].mean()

In [ ]:
for i in unique_complaint:
    for k in set(unique_complaint) - {i,i}:
        print('mean for {} is : {}'.format(i , average_response_time[i]))        
        print('mean for {} is : {}'.format(k , average_response_time[k]))
        
        sat1 = list(project[project['Complaint Type'] == i]['Request_Closing_Time_in_seconds'].dropna())
        sat2 = list(project[project['Complaint Type'] == k]['Request_Closing_Time_in_seconds'].dropna())
        
        test_statistic , p_value = ttest_ind(sat1 , sat2)     
        print('{} --> p value for similar average for {} , {}\n\n'.format(round(p_value,3) , i , k))
 

In [ ]:
# Thank you.